In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [2]:
import pandas as pd

# Carregar os DataFrames a partir dos arquivos CSV
df = pd.read_csv('test.csv')
dft = pd.read_csv('todascenarioRafa.csv')

# Ver as colunas de cada DataFrame
print("Colunas do DataFrame 'df':")
print(df.columns)

print("\nColunas do DataFrame 'dft':")
print(dft.columns)


Colunas do DataFrame 'df':
Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city', 'ID'],
      dtype='object')

Colunas do DataFrame 'dft':
Index(['id_transmissao', 'transmission_type', 'body_type', 'id_body_type',
       'modelo', 'id_modelos', 'marca', 'id_categoria_marca', 'ano', 'city',
       'id_city', 'insurance', 'id_insurance', 'seats', 'engine_capacity',
       'engine_capacity_group', 'id_engine_capacity_group', 'fuel_type',
       'id_fuel_type', 'kms_driven', 'kms_group', 'id_kms_group', 'max_power',
       'max_power_group', 'id_max_power_group', 'owner_type', 'id_owner_type',
       'resale_price_Lakh'],
      dtype='object')


In [3]:
#Antes de mudar formatos

# SEATS __________________________________________________
# Check for model cars with missing values in the 'seats' column
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

# Update number of seats based on conditions
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI']), 'seats'] = 5
df.loc[df['full_name'] == '2008 Ford Endeavour 3.0 4x4 Thunder Plus', 'seats'] = 7

# Check
distinct_full_names_with_blank_seats = df[df['seats'].isnull()]['full_name'].unique()
print()
print('Distinct full names with blank seats:')
for full_name in distinct_full_names_with_blank_seats:
    print(full_name)

# Calculando a moda para 'owner_type'
moda_owner_type = df['seats'].mode()[0]

# Preenchendo os valores nulos com a moda calculada
df['seats'].fillna(moda_owner_type, inplace=True)


# ENGINE __________________________________________________
#remove cc from engine_capacity
df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')

# FUEL TYPE __________________________________________________
# unique 'fuel_type'
valores_unicos_fuel_type = df['fuel_type'].unique()
print()
print(valores_unicos_fuel_type)

# KMS __________________________________________________
#remove kms from kms_driven
df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
# Remove , and blanks from 'kms_driven'
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()

# MAX POWER 
# #convert to bhp__________________________________________________
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)


# Convert not NaN to float
Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]

df['max_power'] = Correct_Max_Power



# Calculando a moda para 'owner_type'
moda_owner_type = df['owner_type'].mode()[0]

# Preenchendo os valores nulos com a moda calculada
df['owner_type'].fillna(moda_owner_type, inplace=True)


# Check
unique_values = df['owner_type'].unique()
print()
print(unique_values)



# INSURANCE __________________________________________________
#Delet "insurance" 
df['insurance'] = df['insurance'].str.replace('insurance', '')

#Assign value to blank lines
df['insurance'].fillna('No Insurance', inplace=True)

#Blank space on "Third Party" lines
df['insurance'] = df['insurance'].str.replace('Third Party ', 'Third Party')

#Remove integer values
df = df[~df['insurance'].astype(str).str.isnumeric()]

#MILEAGE
#assign value to blank lines
df['mileage'] = df['mileage'].fillna(0)

df = df[df['mileage'].notna()]
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+.\d+|\d+)', str(i))  # Encontrar números (com ou sem ponto decimal)

    if match:
        valor = float(match.group())

        if 'km/kg' in str(i):
            valor *= 1.40  # Converter de km/kg para kmpl

        consumo_correto.append(valor)

df['mileage'] = consumo_correto


df = df.astype({
   
    'kms_driven': float,
   
})

Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2009 Maruti Estilo LXI
2012 Honda Jazz Select Edition

Distinct full names with blank seats:
2005 Honda City 1.5 EXI
2012 Honda Jazz Select Edition

['Petrol' 'Diesel' 'CNG' 'Electric' 'LPG']

['First Owner' 'Second Owner' 'Third Owner' 'Fourth Owner' 'Fifth Owner']


In [4]:
#FullName ______________________________________

#ANO
# Extract the year and create a new column
df['ano'] = df['full_name'].str.extract(r'(\d{4})')

# Display the resulting DataFrame
print(df['ano'])



#MODELO
# Get everything that comes after "marca"
df['modelo'] = df['full_name'].str.extract(r'\d{4}\s\w+\s(.+)')

# Print the resulting DataFrame
print(df['modelo'])

0       2017
1       2020
2       2010
3       2010
4       2018
        ... 
3485    2022
3486    2017
3487    2017
3488    2016
3489    2017
Name: ano, Length: 3490, dtype: object
0       Grand i10 1.2 Kappa Sportz Option
1                           Swift AMT VXI
2                     Zen Estilo LX BSIII
3                                Alto LXi
4                     i20 Active S Petrol
                      ...                
3485                     Tigor EV XZ Plus
3486                         KWID 1.0 RXT
3487                       City i VTEC VX
3488                    Jazz 1.2 V i VTEC
3489                            Swift VXI
Name: modelo, Length: 3490, dtype: object


In [6]:
#BODY TYPE ----------------------
#Replace the missing types and the ones that were wrong
df.loc[df['full_name'].isin(['2009 Maruti Estilo LXI', '2019 Nissan Kicks Diesel', '2010 Maruti Estilo LXI', '2019 Maruti Wagon R ZXI AMT 1.2BSIV',
                             '2016 Mercedes-Benz B Class B180 Sport', '2016 Mercedes-Benz A Class A200 D Sport Edition',
                             '2004 Tata Indigo Marina GLX', '2017 Volvo V40 D3 R Design']), 'body_type'] = 'Hatchback'
df.loc[df['full_name'].isin(['2005 Tata Sumo Victa EX 10/7 Str BSII', '2008 Tata Sumo CX', '2010 Tata New Safari DICOR 2.2 EX 4x2', 
                             '2017 Mahindra KUV 100 mFALCON G80 K8', '2019 Mahindra Bolero Camper GOLD ZX 2WD']), 'body_type'] = 'SUV'
df.loc[df['full_name'].isin(['2015 Honda Mobilio E i-DTEC', '2018 Maruti Eeco 5 Seater AC BSIV', '2016 Datsun GO Plus A']), 'body_type'] = 'MUV'
df.loc[df['full_name'].isin(['2021 BMW 7 Series 730Ld DPE Signature','2006 Hyundai Accent CRDi', '2007 Maruti SX4 Vxi BSIII',
                             '2020 Mercedes-Benz S-Class Maybach S560', '2004 Skoda Octavia 1.8 Turbo Petrol RS MT']), 'body_type'] = 'Sedan'
df.loc[df['full_name'].isin(['2012 Mercedes-Benz SLK-Class SLK 350', '2013 Mercedes-Benz E-Class E350 Cabriolet', 
                             '2009 Mercedes-Benz SLK-Class 55 AMG']), 'body_type'] = 'Convertibles'
df.loc[df['full_name'] == '2011 Jaguar XK R V8 Coupe', 'body_type'] = 'Coupe'
df.loc[df['full_name'].isin(['2015 Chevrolet Enjoy 1.4 LS 7', '2004 Maruti Versa DX2 8-SEATER BSIII TWIN A/C', '2021 Maruti Eeco 5 STR With AC Plus HTR CNG',
                             '2022 Maruti Eeco 5 Seater AC 2020-2022', '2022 Maruti Eeco 5 Seater AC BSVI', '2017 Maruti Eeco Flexi Green', 
                             '2013 Maruti Omni Limited Edition', '2021 Maruti Eeco CNG 5 Seater AC', '2019 Maruti Eeco CNG 5 Seater AC BSIV', 
                             '2019 Maruti Eeco 5 Seater AC', '2013 Maruti Eeco 5 Seater Standard BSIV', '2015 Maruti Omni CNG', 
                             '2015 Maruti Omni MPI CARGO BSIV', '2016 Maruti Eeco CNG 5 Seater AC BSIV', '2013 Toyota Innova 2.5 E Diesel MS 7-seater',
                             '2015 Chevrolet Enjoy 1.4 LS 7']), 'body_type'] = 'Minivans'
df.loc[df['full_name'].isin(['2014 Tata Xenon XT EX 4X2', '2019 Isuzu D-Max High']), 'body_type'] = 'Pickup'

# Check
distinct_full_names_with_blank_body_type = df[df['body_type'].isnull()]['full_name'].unique()
print('Distinct full names with blank body_type:')
for full_name in distinct_full_names_with_blank_body_type:
    print(full_name)

Distinct full names with blank body_type:


In [7]:
# Mudar formatos

df = df.astype({
    'seats': float,
    'engine_capacity': float,
    'kms_driven': float,
    'max_power': float,
    'ano': float,
   
    'modelo': 'string',
    'body_type': 'string',
    'mileage': float,
})

In [8]:
import pandas as pd

# Calculando a média para 'kms_driven' e 'max_power'
media_kms = df['kms_driven'].mean()
media_max_power = df['max_power'].mean()

# Preenchendo os valores nulos com as médias calculadas
df['kms_driven'].fillna(media_kms, inplace=True)
df['max_power'].fillna(media_max_power, inplace=True)


In [9]:
import pandas as pd
import re

# Supondo que já carregou os DataFrames df e dft

# Função para extrair a marca a partir da coluna 'full_name' em df
def extrair_marca(nome):
    match = re.search(r'\d{4}\s(.+?)\s', nome)  # Expressão regular para pegar a marca após o ano
    if match:
        return match.group(1)
    else:
        return None

# Aplicar a função para extrair a marca e criar uma nova coluna 'marca' em df
df['marca'] = df['full_name'].apply(extrair_marca)

# Mapear as marcas de df para a coluna 'marca' em dft
mapeamento_marca = dict(zip(dft['marca'], dft['id_categoria_marca']))

# Criar uma nova coluna 'id_categoria_marca' em df com base na correspondência de marcas em dft
df['id_categoria_marca'] = df['marca'].map(mapeamento_marca)

# Visualizar o DataFrame df atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity      insurance transmission_type  kms_driven  \
0              1197.0  Comprehensive            Manual     53658.

In [10]:
import pandas as pd

def adicionar_0_1_maior_valor(df, dft):
    # Encontrar o maior valor na coluna 'id_categoria_marca' em dft
    maior_valor_categoria_marca_dft = dft['id_categoria_marca'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_categoria_marca_dft + 0.1

    # Substituir os valores NaN na coluna 'id_categoria_marca' em df pelo novo valor
    df['id_categoria_marca'].fillna(novo_valor, inplace=True)

# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valor(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity      insurance transmission_type  kms_driven  \
0              1197.0  Comprehensive            Manual     53658.

In [11]:
# Definir os limites dos grupos de kms_group
limites = [300.0, 200250.0, 400200.0, 600150.0, 800100.0, 1000050.0, np.inf]

# Rótulos dos grupos
rotulos = [
    '300.0-200250.0 KMS',
    '200250.0-400200.0 KMS',
    '400200.0-600150.0 KMS',
    '600150.0-800100.0 KMS',
    '800100.0-1000050.0 KMS',
    '1000050.0-inf KMS'
]

# Criar a coluna 'kms_group' com base nos limites e rótulos definidos
df['kms_group'] = pd.cut(df['kms_driven'], bins=limites, labels=rotulos, right=False)

# Exibir o DataFrame com a coluna 'kms_group' atualizada
print(df)






                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity      insurance transmission_type  kms_driven  \
0              1197.0  Comprehensive            Manual     53658.

In [12]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['kms_group'], dft['id_kms_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_kms_group'] = df['kms_group'].map(mapeamento_max_power_group)

In [13]:
# Definir os limites dos grupos de max_power_group
limites_max_power = [25.4, 113.17, 200.93, 288.7, 376.47, 464.23, np.inf]

# Rótulos dos grupos
rotulos_max_power = [
    '25.4-113.17 bhp',
    '113.17-200.93 bhp',
    '200.93-288.7 bhp',
    '288.7-376.47 bhp',
    '376.47-464.23 bhp',
    '464.23-inf bhp'
]

# Criar a coluna 'max_power_group' com base nos limites e rótulos definidos
df['max_power_group'] = pd.cut(df['max_power'], bins=limites_max_power, labels=rotulos_max_power, right=False)

# Exibir o DataFrame com a coluna 'max_power_group' atualizada
print(df)


                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity      insurance transmission_type  kms_driven  \
0              1197.0  Comprehensive            Manual     53658.

In [14]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['max_power_group'], dft['id_max_power_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_max_power_group'] = df['max_power_group'].map(mapeamento_max_power_group)

In [18]:
# Definir os limites dos grupos de max_power_group
limites_engine_capacity_group = [0.0, 999.67, 1999.33, 2999.0, 3998.67, 4998.33, np.inf]

# Rótulos dos grupos de max_power_group
rotulos_engine_capacity_group = [
    '0.0-999.67 cc',
    '999.67-1999.33 cc',
    '1999.33-2999.0 cc',
    '2999.0-3998.67 cc',
    '3998.67-4998.33 cc',
    '4998.33-inf cc'
]

# Criar a coluna 'max_power_group' com base nos limites e rótulos definidos
df['engine_capacity_group'] = pd.cut(df['engine_capacity'], bins=limites_engine_capacity_group, labels=rotulos_engine_capacity_group, right=False)

# Exibir o DataFrame com a coluna 'max_power_group' atualizada
print(df)


                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity      insurance transmission_type  kms_driven  \
0              1197.0  Comprehensive            Manual     53658.

In [19]:
# Criar um dicionário de mapeamento de 'max_power_group' para 'id_max_power_group' em dft
mapeamento_max_power_group = dict(zip(dft['engine_capacity_group'], dft['id_engine_capacity_group']))

# Criar uma nova coluna 'id_max_power_group' em df com base na correspondência de 'max_power_group' em dft
df['id_engine_capacity_group'] = df['engine_capacity_group'].map(mapeamento_max_power_group)

In [21]:
# Substituir os valores 'Third Party insurance' por 'Third Party' em df
df['insurance'] = df['insurance'].replace('Third Party insurance', 'Third Party', regex=True)

# Substituir os valores NaN por 'No Insurance' em df
df['insurance'].fillna('No Insurance', inplace=True)

# Visualizar a coluna 'insurance' atualizada em df
print(df['insurance'].unique())
print(dft['id_insurance'].unique())

['Comprehensive' 'Not Available' 'Third Party' 'Zero Dep' 'No Insurance']
[0.25493928 1.31039371 0.44796011 1.41423674 3.35375599]


In [23]:
# Mapear os valores desejados em 'df' para os valores correspondentes em 'dft'
mapeamento_valores = {
    'Third Party insurance': 'Third Party',
    pd.NA: 'No Insurance'
}

# Aplicar as transformações nos valores da coluna 'insurance' em 'df'
df['insurance'] = df['insurance'].replace(mapeamento_valores).fillna('No Insurance')

# Visualizar os valores únicos após as transformações em 'df'
print(df['insurance'].unique())

['Comprehensive' 'Not Available' 'Third Party' 'Zero Dep' 'No Insurance']


In [25]:
import pandas as pd


def substituir_valores(df, coluna_alvo, mapeamento):
    df[coluna_alvo] = df[coluna_alvo].astype(str)
    df[coluna_alvo] = df[coluna_alvo].replace(mapeamento).fillna(df[coluna_alvo])


# Criar os mapeamentos desejados para cada coluna
mapeamento_id_cidade = dict(zip(dft['city'], dft['id_city']))
mapeamento_id_body_type = dict(zip(dft['body_type'], dft['id_body_type']))
mapeamento_id_owner_type = dict(zip(dft['owner_type'], dft['id_owner_type']))
mapeamento_id_fuel_type = dict(zip(dft['fuel_type'], dft['id_fuel_type']))
mapeamento_id_insurance = dict(zip(dft['insurance'], dft['id_insurance']))
mapeamento_id_transmissao = dict(zip(dft['transmission_type'], dft['id_transmissao']))


# Chamar a função para substituir os valores nas respectivas colunas
substituir_valores(df, 'city', mapeamento_id_cidade)
substituir_valores(df, 'body_type', mapeamento_id_body_type)
substituir_valores(df, 'owner_type', mapeamento_id_owner_type)
substituir_valores(df, 'fuel_type', mapeamento_id_fuel_type)
substituir_valores(df, 'insurance', mapeamento_id_insurance)
substituir_valores(df, 'transmission_type', mapeamento_id_transmissao)

# Visualizar o DataFrame 'df' atualizado
print(df)



                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance  transmission_type  kms_driven  owner_type  \
0              1197.0   0.447960           0.130520     

In [30]:

#soluçoes id_body_type

import pandas as pd

def adicionar_0_1_maior_valormax(df, dft):
    # Encontrar o maior valor na coluna 'id_body_type' em dft
    maior_valor_id_max_power_dft = dft['id_body_type'].max()

    # Adicionar 0.1 ao maior valor encontrado
    novo_valor = maior_valor_id_max_power_dft + 0.1

    # Valores a substituir e seus respectivos substitutos
    substitute_values = {
        'Tata': novo_valor,
        'Mahindra': novo_valor,
        'Maruti': novo_valor,
        'Audi': novo_valor,
        'Toyota': novo_valor,
        'Honda': novo_valor,
        'BMW': novo_valor,
        'Datsun': novo_valor,
        'Chevrolet': novo_valor,
        'Jaguar': novo_valor,
        'Porsche': novo_valor
    }

    # Substituir os valores na coluna 'body_type' pelos valores do dicionário substitute_values
    df['body_type'].replace(substitute_values, inplace=True)



# Utilização da função com os DataFrames 'df' e 'dft'
adicionar_0_1_maior_valormax(df, dft)

# Visualizar o DataFrame 'df' atualizado
print(df)

                                           full_name registered_year  \
0     2017 Hyundai Grand i10 1.2 Kappa Sportz Option        Jun 2017   
1                          2020 Maruti Swift AMT VXI        Feb 2021   
2                    2010 Maruti Zen Estilo LX BSIII        Jul 2010   
3                               2010 Maruti Alto LXi            2010   
4                   2018 Hyundai i20 Active S Petrol        Oct 2018   
...                                              ...             ...   
3485                      2022 Tata Tigor EV XZ Plus            2022   
3486                       2017 Renault KWID 1.0 RXT        Apr 2017   
3487                       2017 Honda City i VTEC VX        Dec 2017   
3488                    2016 Honda Jazz 1.2 V i VTEC        Jun 2016   
3489                           2017 Maruti Swift VXI        Mar 2017   

      engine_capacity  insurance  transmission_type  kms_driven  owner_type  \
0              1197.0   0.447960           0.130520     

In [31]:
# Obtendo os valores únicos da coluna 'body_type'
valores_unicos_body_type = df['body_type'].unique()

# Exibindo os valores únicos
print(valores_unicos_body_type)


[0.36289937 0.60030061 3.75478599 0.5783883  1.34943462 2.2868092
 3.2868092  3.052726   3.43293724]


In [32]:
# Contar a quantidade de NaN em cada coluna do DataFrame X
nan_count = df.isna().sum()

# Exibir a contagem de NaN por coluna
print(nan_count)

full_name                    0
registered_year             15
engine_capacity              5
insurance                    0
transmission_type            0
kms_driven                   0
owner_type                   0
fuel_type                    0
max_power                    0
seats                        0
mileage                      0
body_type                    0
city                         0
ID                           0
ano                          0
modelo                      73
marca                        0
id_categoria_marca           0
kms_group                    0
id_kms_group                 0
max_power_group              0
id_max_power_group           0
engine_capacity_group        5
id_engine_capacity_group     0
dtype: int64


In [33]:
df = df.astype({
    
    'ano': float,
    'body_type': float,
    
  
})

In [47]:
# Suponha que você queira remover as colunas 'coluna1', 'coluna2' e 'coluna3' do DataFrame 'df'
colunas_para_drop = ['kms_group']

# Utilize o método drop para remover as colunas desejadas
df = df.drop(columns=colunas_para_drop)




In [48]:
df

,insurance,transmission_type,owner_type,fuel_type,max_power,seats,mileage,body_type,city,ID,ano,id_categoria_marca,id_kms_group,id_max_power_group,id_engine_capacity_group
0,0.447960,0.130520,0.151222,0.180375,81.8,5.0,18.90,0.362899,0.9253511986753568,0,2017.0,0.073937,0.000706,0.146167,0.126083
1,1.414237,0.585488,0.620425,0.180375,81.8,5.0,21.21,0.362899,1.2812610075913198,1,2020.0,0.073937,0.000706,0.146167,0.126083
2,0.254939,0.130520,1.354480,0.180375,67.1,5.0,18.20,0.362899,1.5928311439816096,2,2010.0,0.073937,0.000706,0.146167,0.769991
3,0.254939,0.130520,0.151222,0.180375,46.3,5.0,19.70,0.362899,0.7696133048101551,3,2010.0,0.073937,0.000706,0.146167,0.769991
4,0.447960,0.130520,0.620425,0.180375,81.8,5.0,17.19,0.362899,0.7696133048101551,4,2018.0,0.073937,0.000706,0.146167,0.126083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3485,0.254939,0.585488,0.151222,2.450666,73.7,5.0,0.00,0.578388,1.0924931235982491,3485,2022.0,0.073937,0.000706,0.146167,4.131907
3486,0.254939,0.130520,0.151222,0.180375,67.0,5.0,23.01,0.362899,0.8694561530439567,3486,2017.0,0.073937,0.000706,0.146167,0.769991
3487,0.447960,0.130520,0.151222,0.180375,117.3,5.0,17.80,0.578388,0.7696133048101551,3487,2017.0,0.073937,0.000706,0.576088,0.126083
3488,0.447960,0.130520,0.151222,0.180375,88.7,5.0,18.70,0.362899,0.9253511986753568,3488,2016.0,0.073937,0.000706,0.146167,0.126083


In [50]:
#Export
#Not the target
colunas_selecionadas = ['body_type','id_categoria_marca','ano','city','insurance','transmission_type','seats','mileage','id_engine_capacity_group','fuel_type','id_kms_group','id_max_power_group','owner_type','ID']

if set(colunas_selecionadas).issubset(df.columns):
    df_selecionado = df[colunas_selecionadas]
    df_selecionado.to_csv('testcenario2Rafa.csv', index=False)

In [51]:
df_colunas_selecionadas = pd.DataFrame(colunas_selecionadas)

# Contar a quantidade de NaN em cada coluna do DataFrame df_colunas_selecionadas
nan_count = df_colunas_selecionadas.isna().sum()

# Exibir a contagem de NaN por coluna
print(nan_count)


0    0
dtype: int64
